In [5]:
import cv2 as cv
import numpy as np
from flask import Flask,render_template 
from keras import models 

In [7]:
model=models.load_model('face_mask_detector_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
label={0:'mask',1:'no mask'}
face_classifier=cv.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

In [ ]:
def detect1(frame):
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.2)
    for a,b,c,d in faces:
        crop=frame[b:b+d,a:a+c]
        resized=cv.resize(crop,(224,224))
        resized=resized/255.0
        resized=np.expand_dims(resized,axis=0)
        pred=model.predict(resized)
        prediction=np.argmax(pred,axis=1)[0]
        pct=str(int(np.max(pred,axis=1)[0]*100))
        if prediction==0:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,255,0),3)
            cv.putText(frame,label[0]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,255,0),2)
        else:
            cv.rectangle(frame,(a,b),(a+c,b+d),(0,0,255),3)
            cv.putText(frame,label[1]+pct+'%',(a,b-10),cv.FONT_HERSHEY_SIMPLEX,.8,(0,0,255),2)
    return frame
            
app=Flask(__name__,template_folder='template')
@app.route('/')
def index():
    return render_template('index.html')
@app.route('/prediction',methods=['POST','GET'])
def pre():
    c=cv.VideoCapture(0)
    while True:
        ret,frame=c.read()
        cv.imshow('live mask prediction',detect1(frame))
        if cv.waitKey(1)==13:
            break        
    c.release()
    cv.destroyAllWindows()
    return render_template('index.html')
    
    
    
    
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2020 00:28:24] "GET / HTTP/1.1" 200 -
[2020-10-23 00:28:41,716] ERROR in app: Exception on /prediction [GET]
Traceback (most recent call last):
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\ajay\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.view_function